# Verification Simulation Setup
In this section, we import the required simulation and visualization modules used for validating the optimized nanofocusing structure. These include Meep for FDTD simulation, the adjoint module for handling field calculations, Autograd for numerical operations, and Matplotlib for plotting. We also define the material refractive indices that will appear in the verification environment.
本段程式碼負責匯入進行奈米聚焦結構驗證所需的主要套件，包括 Meep 的 FDTD 模擬工具、adjoint 模組的場計算功能、Autograd 的數值運算工具，以及 Matplotlib 的繪圖函式。

In [ ]:
import meep as mp
import meep.adjoint as mpa
from meep import Animate2D
import numpy as np
from autograd import numpy as npa
from autograd import tensor_jacobian_product, grad
import nlopt
from matplotlib import pyplot as plt
from matplotlib.patches import Circle
from meep.materials import Ag
import os
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from PIL import Image
import matplotlib.colors as mcolors

mp.verbosity(1)
TiO2 = mp.Medium(index=2.6)
SiO2 = mp.Medium(index=1.44)
Si   = mp.Medium(index=3.4)
Air  = mp.Medium(index=1)

## Data Folder Setup / 資料夾建立
本段程式碼預先建立多個資料夾，用來分類儲存後續模擬與後處理產生的 DFT 資料與分析結果，包括：

DFT_GAS：儲存一般模擬下取得的 DFT 場資料（如 |E|²、穿透等）。
DFT_mode_volume：儲存計算模態體積（mode volume）相關的數據。
DFT_mode_volume_emp：儲存空場或比較用的模態體積資料。
DFT_empty_GAS：儲存空腔／空場（無結構）模擬下的 DFT 資料，用於正規化。
DFT_charge_density：儲存電荷密度分佈或相關分析結果。
透過事先建立這些資料夾，可以讓後續輸出的檔案自動分門別類，方便管理

In [ ]:
os.makedirs('DFT_GAS',exist_ok=True) # Create Folder
os.makedirs('DFT_mode_volume',exist_ok=True) # Create Folder
os.makedirs('DFT_mode_volume_emp',exist_ok=True) # Create Folder
os.makedirs('DFT_empty_GAS',exist_ok=True) # Create Folder
os.makedirs('DFT_charge_density',exist_ok=True) # Create Folder

## Simulation Setup for Verification
本段為驗證用模擬環境的建立。所有材料、解析度、模擬範圍、光源與監測設定皆與拓樸優化程式保持一致，確保最終的場分布、增強效果與模擬結果能與優化階段進行正確比對。

In [ ]:
resolution = 100
design_region_resolution = int(resolution)

design_region_x_width  = 1   #100 nm
design_region_y_height = 1   #100 nm
design_region_z_height = 0.02  #20 nm or 10nm

pml_size = 1.0
pml_layers = [mp.PML(pml_size,direction=mp.Z)]

Sz_size = 0.6
Sx = design_region_x_width
Sy = design_region_y_height 
Sz = 2 * pml_size + design_region_z_height + Sz_size
cell_size = mp.Vector3(Sx, Sy, Sz)

wavelengths = np.array([1.55])     # wavelengths = np.array([1.5 ,1.55, 1.6])
frequencies = np.array([1 / 1.55])

nf = 1                 #3 #wavelengths Number

minimum_length = 0.02  # minimum length scale (microns)
eta_i = 0.5            # blueprint (or intermediate) design field thresholding point (between 0 and 1)
eta_e = 0.55           # erosion design field thresholding point (between 0 and 1)
eta_d = 1 - eta_e      # dilation design field thresholding point (between 0 and 1)
filter_radius = mpa.get_conic_radius_from_eta_e(minimum_length, eta_e)

Source_distance = -0.2

fcen   = 1 / 1.55
width  = 0.2  
fwidth = width * fcen
source_center = mp.Vector3(0,0,Source_distance)  
source_size   = mp.Vector3(design_region_x_width, design_region_y_height, 0)
src    = mp.GaussianSource(frequency=fcen, fwidth=fwidth, is_integrated=True)
source = [mp.Source(src, component=mp.Ey, size=source_size, center=source_center)]   

Nx = int(design_region_resolution * design_region_x_width) + 1
Ny = int(design_region_resolution * design_region_y_height) + 1
Nz = 1

design_variables = mp.MaterialGrid(mp.Vector3(Nx, Ny, Nz), Air, Ag, grid_type="U_MEAN")
design_region    = mpa.DesignRegion(
            design_variables,
            volume=mp.Volume(
            center=mp.Vector3(0,0,0),
            size=mp.Vector3(design_region_x_width, design_region_y_height, design_region_z_height),
            ),
)

def mapping(x, eta, beta):
    # filter
    filtered_field = mpa.conic_filter(
        x,
        filter_radius,
        design_region_x_width,
        design_region_y_height,
        design_region_resolution,
    )
    # projection
    projected_field = mpa.tanh_projection(filtered_field, beta, eta)
    projected_field = (npa.fliplr(projected_field) + projected_field) / 2
    projected_field = (npa.flipud(projected_field) + projected_field) / 2  # left-right symmetry    
    
    return projected_field.flatten()


geometry = [mp.Block(center=design_region.center, size=design_region.size, material=design_variables)]

kpoint = mp.Vector3()
sim = mp.Simulation(
    cell_size        = cell_size,
    boundary_layers  = pml_layers,
    geometry         = geometry,
    sources          = source,
    default_material = Air,
    k_point          = kpoint,
    symmetries       = [mp.Mirror(direction=mp.X)],
    resolution       = resolution,
    extra_materials  = [Ag],       # Introducing metal complex terms
)
#---------------------------------------------------------------------------------------------------#


#-----------------[4]---------------------------------#
monitor_position   = mp.Vector3(0, 0, 0)       # Focus position
monitor_size       = mp.Vector3(0.01,0.01,design_region_z_height)     # Focus Size//////0.11
FourierFields_x    = mpa.FourierFields(sim,mp.Volume(center=monitor_position,size=monitor_size),mp.Ex,yee_grid=True)
FourierFields_y    = mpa.FourierFields(sim,mp.Volume(center=monitor_position,size=monitor_size),mp.Ey,yee_grid=True)
FourierFields_z    = mpa.FourierFields(sim,mp.Volume(center=monitor_position,size=monitor_size),mp.Ez,yee_grid=True)
ob_list            = [FourierFields_x,FourierFields_y,FourierFields_z]


def J(fields_x,fields_y,fields_z):
    ET_x = 0
    ET_y = 0
    ET_z = 0
    ET_x_length = fields_x.shape[1]
    ET_y_length = fields_y.shape[1]
    ET_z_length = fields_z.shape[1]
    for k in range(0,ET_x_length):
        ET_x = ET_x + npa.abs(fields_x[:,k]) ** 2
    for k in range(0,ET_y_length):
        ET_y = ET_y + npa.abs(fields_y[:,k]) ** 2
    for k in range(0,ET_z_length):
        ET_z = ET_z + npa.abs(fields_z[:,k]) ** 2
    ET = ( (npa.mean(ET_x)) + (npa.mean(ET_y)) + (npa.mean(ET_z)) ) ** (1/2)  
    return ET


opt = mpa.OptimizationProblem(
    simulation=sim,
    objective_functions=[J],
    objective_arguments=ob_list,
    design_regions=[design_region],
    frequencies=frequencies,
    decimation_factor = 1 ,           # KEY BUG!!
    maximum_run_time=50,
)

## Load Optimized Design
本段將讀取拓樸優化最終輸出的資料，並更新至模擬中的設計區域，以便進行電磁場驗證與分析。

In [ ]:
evaluation_history = np.load('Post_evaluation_history.npy')
beta_A  = np.load("Post_beta_scale_array.npy")
eta_A   = np.load("Post_eta_i_array.npy")
cur_A   = np.load("Post_cur_beta_array.npy")
x_A     = np.load("Post_x_array.npy")
opt.update_design([mapping(x_A, eta_A, cur_A/beta_A)])

## Field Sampling and Data Export  
## 電場取樣與資料輸出

In this section, we reuse the optimized design and rebuild the simulation to sample the electromagnetic fields of the final nanofocusing structure. Several DFT monitors are added to record the complex field distributions (Ex, Ey, Ez) on different cross-sections and volumes of the domain, including:

- x–z and y–z planes for visualizing the focusing behavior along the propagation direction.
- x–y planes at two different z-positions near the metal film for near-field pattern analysis.
- A 3D volume covering the design region and surrounding space for mode-volume calculation.
- Yee-grid based volumes and planes for evaluating charge-density related quantities above and below the metal film.

After running the simulation until the DFT fields decay, the corresponding field arrays are extracted via `get_dft_array()` and saved as `.npy` files into dedicated folders (`DFT_GAS`, `DFT_mode_volume`, `DFT_charge_density`). These stored datasets are later used for post-processing, including nanofocusing visualization, localization length estimation, mode-volume computation, and charge-density analysis.

本段程式碼在套用最終優化結構後，重新建立模擬並對電磁場進行取樣。程式會在多個位置與體積加入 DFT 監測器，以記錄最終結構的複數電場分佈 (Ex, Ey, Ez)，包含：

- 沿傳播方向的 x–z 與 y–z 剖面，用於觀察聚焦行為。
- 接近金屬薄膜的兩個 x–y 平面，用於分析近場場型。
- 覆蓋設計區與周圍空間的三維體積，用於計算模態體積。
- 使用 Yee 格點的體積與平面，用於評估金屬上下方的電荷密度相關量。

模擬在頻域場能量衰減後結束，接著利用 `get_dft_array()` 讀出各監測器的場分佈，並以 `.npy` 檔案形式分別存入 `DFT_GAS`、`DFT_mode_volume` 與 `DFT_charge_density` 資料夾。這些資料將用於後續的後處理分析，例如奈米聚焦視覺化、局域化長度估算、模態體積計算以及電荷密度分佈研究。


In [ ]:
opt.sim = mp.Simulation(
    cell_size=mp.Vector3(Sx, Sy ,Sz),
    boundary_layers=pml_layers,
    geometry=geometry,
    sources=source,
    default_material=Air,
    resolution=resolution,
    k_point=kpoint,
    #symmetries=[mp.Mirror(direction=mp.X)],
    extra_materials = [Ag],
)
#src = mp.ContinuousSource(frequency=frequencies[0], fwidth=0.01 ,is_integrated=True)
src    = mp.GaussianSource(frequency=fcen, fwidth=fwidth, is_integrated=True)
source = [mp.Source(src, component=mp.Ey, size=source_size, center=source_center)] 
opt.sim.change_sources(source)

dft_fields_xz = opt.sim.add_dft_fields([mp.Ex,mp.Ey,mp.Ez],
                                fcen,0,1,
                                center=mp.Vector3(),
                                size=mp.Vector3(Sx,0,Sz-2 * pml_size),
                                yee_grid=False
                                )

dft_fields_yz = opt.sim.add_dft_fields([mp.Ex,mp.Ey,mp.Ez],
                                fcen,0,1,
                                center=mp.Vector3(),
                                size=mp.Vector3(0,Sy,Sz-2 * pml_size),
                                yee_grid=False
                                )

dft_fields_xyz12 = opt.sim.add_dft_fields([mp.Ex,mp.Ey,mp.Ez],
                                fcen,0,1,
                                center=mp.Vector3(0,0,0.01),
                                size=mp.Vector3(Sx,Sy,0),
                                yee_grid=False
                                )

dft_fields_xyz00 = opt.sim.add_dft_fields([mp.Ex,mp.Ey,mp.Ez],
                                fcen,0,1,
                                center=mp.Vector3(0,0,0),
                                size=mp.Vector3(Sx,Sy,0),
                                yee_grid=False 
                                )

dft_fields_x0yz0 = opt.sim.add_dft_fields([mp.Ex,mp.Ey,mp.Ez],
                                fcen,0,1,
                                center=mp.Vector3(0,0,0.01),
                                size=mp.Vector3(0,Sy,0),
                                yee_grid=False
                                )

dft_fields_mode_volume = opt.sim.add_dft_fields([mp.Ex,mp.Ey,mp.Ez],
                                fcen,0,1,
                                center=mp.Vector3(0,0,0),
                                size=mp.Vector3(Sx,Sy,design_region_z_height + Sz_size),
                                yee_grid=False 
                                )

dft_fields_CD_all = opt.sim.add_dft_fields([mp.Ex,mp.Ey,mp.Ez],
                                fcen,0,1,
                                center=mp.Vector3(0,0,0),
                                size=mp.Vector3(Sx,Sy,design_region_z_height + Sz_size),
                                yee_grid=True 
                                )

dft_fields_CD_pos = opt.sim.add_dft_fields([mp.Ex,mp.Ey,mp.Ez],
                                fcen,0,1,
                                center=mp.Vector3(0,0,design_region_z_height/2+0.01),
                                size=mp.Vector3(Sx,Sy,0),
                                yee_grid=True 
                                )

dft_fields_CD_neg = opt.sim.add_dft_fields([mp.Ex,mp.Ey,mp.Ez],
                                fcen,0,1,
                                center=mp.Vector3(0,0,-(design_region_z_height/2+0.01)),
                                size=mp.Vector3(Sx,Sy,0),
                                yee_grid=True 
                                )


#opt.sim.run(until_after_sources = 0)
opt.sim.run(until_after_sources=mp.stop_when_dft_decayed(
            tol=1e-11,
            minimum_run_time=0,
            maximum_run_time=50))


# In[8]:


Ex_xz = opt.sim.get_dft_array(dft_fields_xz,mp.Ex,0)
Ey_xz = opt.sim.get_dft_array(dft_fields_xz,mp.Ey,0)
Ez_xz = opt.sim.get_dft_array(dft_fields_xz,mp.Ez,0)
np.save('DFT_GAS/Dft_GAS_Ex_xz.npy',Ex_xz)
np.save('DFT_GAS/Dft_GAS_Ey_xz.npy',Ey_xz)
np.save('DFT_GAS/Dft_GAS_Ez_xz.npy',Ez_xz)

Ex_yz = opt.sim.get_dft_array(dft_fields_yz,mp.Ex,0)
Ey_yz = opt.sim.get_dft_array(dft_fields_yz,mp.Ey,0)
Ez_yz = opt.sim.get_dft_array(dft_fields_yz,mp.Ez,0)
np.save('DFT_GAS/Dft_GAS_Ex_yz.npy',Ex_yz)
np.save('DFT_GAS/Dft_GAS_Ey_yz.npy',Ey_yz)
np.save('DFT_GAS/Dft_GAS_Ez_yz.npy',Ez_yz)

Ex01 = opt.sim.get_dft_array(dft_fields_xyz12,mp.Ex,0)
Ey01 = opt.sim.get_dft_array(dft_fields_xyz12,mp.Ey,0)
Ez01 = opt.sim.get_dft_array(dft_fields_xyz12,mp.Ez,0)
np.save('DFT_GAS/Dft_GAS_Ex_xyz1.npy',Ex01)
np.save('DFT_GAS/Dft_GAS_Ey_xyz1.npy',Ey01)
np.save('DFT_GAS/Dft_GAS_Ez_xyz1.npy',Ez01)

Ex00 = opt.sim.get_dft_array(dft_fields_xyz00,mp.Ex,0)
Ey00 = opt.sim.get_dft_array(dft_fields_xyz00,mp.Ey,0)
Ez00 = opt.sim.get_dft_array(dft_fields_xyz00,mp.Ez,0)
np.save('DFT_GAS/Dft_GAS_Ex_xyz0.npy',Ex00)
np.save('DFT_GAS/Dft_GAS_Ey_xyz0.npy',Ey00)
np.save('DFT_GAS/Dft_GAS_Ez_xyz0.npy',Ez00)

Ex_mode_volume = opt.sim.get_dft_array(dft_fields_mode_volume,mp.Ex,0)
Ey_mode_volume = opt.sim.get_dft_array(dft_fields_mode_volume,mp.Ey,0)
Ez_mode_volume = opt.sim.get_dft_array(dft_fields_mode_volume,mp.Ez,0)
np.save('DFT_mode_volume/DFT_mode_volume_Ex_xyz0.npy',Ex_mode_volume)
np.save('DFT_mode_volume/DFT_mode_volume_Ey_xyz0.npy',Ey_mode_volume)
np.save('DFT_mode_volume/DFT_mode_volume_Ez_xyz0.npy',Ez_mode_volume)

Ex_CD_all = opt.sim.get_dft_array(dft_fields_CD_all,mp.Ex,0)
Ey_CD_all = opt.sim.get_dft_array(dft_fields_CD_all,mp.Ey,0)
Ez_CD_all = opt.sim.get_dft_array(dft_fields_CD_all,mp.Ez,0)
np.save('DFT_charge_density/DFT_charge_density_Ex_all.npy',Ex_CD_all)
np.save('DFT_charge_density/DFT_charge_density_Ey_all.npy',Ey_CD_all)
np.save('DFT_charge_density/DFT_charge_density_Ez_all.npy',Ez_CD_all)

Ez_CD_pos = opt.sim.get_dft_array(dft_fields_CD_pos,mp.Ez,0)
np.save('DFT_charge_density/DFT_charge_density_Ez_pos.npy',Ez_CD_pos)
Ez_CD_neg = opt.sim.get_dft_array(dft_fields_CD_neg,mp.Ez,0)
np.save('DFT_charge_density/DFT_charge_density_Ez_neg.npy',Ez_CD_neg)

opt.sim.reset_meep()

## Field Data Extraction for Optimized and Reference Simulations  
## 優化結構與空場模擬的電場資料輸出

在完成拓樸優化之後，我們先讀入最終設計結果（`Post_x_array.npy`、`Post_eta_i_array.npy`、`Post_cur_beta_array.npy`、`Post_beta_scale_array.npy` 等），並使用 `opt.update_design()` 將優化後的結構寫回到 `MaterialGrid`。接著，我們重新建立一個以最終結構為幾何的模擬，並在不同的截面與體積區域上加入多組 DFT field monitors，用來輸出電場頻域資料：

- `dft_fields_xz` / `dft_fields_yz`：在 x–z 與 y–z 平面上記錄 Ex, Ey, Ez 的空間分佈，用來觀察縱向與橫向的聚焦情形。
- `dft_fields_xyz12` / `dft_fields_xyz00`：在結構上下不同 z 位置的 x–y 平面上輸出 Ex, Ey, Ez，用於分析焦點附近的橫截面場型。
- `dft_fields_mode_volume`：在涵蓋設計區與周圍空間的三維體積中記錄 Ex, Ey, Ez，用來計算模式體積與能量分佈。
- `dft_fields_CD_all` / `dft_fields_CD_pos` / `dft_fields_CD_neg`：在 Yee grid 上輸出三維與上下表面的 Ez 場，用於後續計算等效表面電荷密度與其分佈。

模擬時間由 `mp.stop_when_dft_decayed()` 自動判斷收斂，在場量衰減到指定誤差後停止。之後，我們藉由 `opt.sim.get_dft_array()` 讀出各 monitor 的 Ex, Ey, Ez 資料，並依用途分別存成 `.npy` 檔：

- 儲存在 `DFT_GAS/` 的檔案：對應有結構時，不同截面上的頻域電場分佈。
- 儲存在 `DFT_mode_volume/` 的檔案：用於計算模式體積與能量集中度。
- 儲存在 `DFT_charge_density/` 的檔案：用於估算表面電荷密度與正負電荷分佈。

接著，我們將同一個模擬設定改為 **沒有任何結構（`geometry=[]`）的空場模擬**，保留相同的來源、網格與 PML 參數，再以相同方式加入 DFT field monitors 並輸出電場：

- 儲存在 `DFT_empty_GAS/` 的檔案：為「無結構」條件下的基準電場分佈。
- 儲存在 `DFT_mode_volume_emp/` 的檔案：為空場條件下的體積場分佈。

這些空場資料將作為後續分析中的正規化基準，用來與「有結構」結果相比較，例如計算電場增強倍率（field enhancement factor）、模式體積縮小倍率以及電荷分佈的相對變化。


In [ ]:
opt.sim = mp.Simulation(
    cell_size=mp.Vector3(Sx, Sy ,Sz),
    boundary_layers=pml_layers,
    geometry=[],
    sources=source,
    default_material=Air,
    resolution=resolution,
    k_point=kpoint,
    #symmetries=[mp.Mirror(direction=mp.X)],
    extra_materials = [Ag],
)
#src = mp.ContinuousSource(frequency=frequencies[0], fwidth=0.01 ,is_integrated=True)
src    = mp.GaussianSource(frequency=fcen, fwidth=fwidth, is_integrated=True)
source = [mp.Source(src, component=mp.Ey, size=source_size, center=source_center)] 
opt.sim.change_sources(source)

dft_fields_xz_emp = opt.sim.add_dft_fields([mp.Ex,mp.Ey,mp.Ez],
                                fcen,0,1,
                                center=mp.Vector3(),
                                size=mp.Vector3(Sx,0,Sz-2 * pml_size),
                                yee_grid=False
                                )

dft_fields_yz_emp = opt.sim.add_dft_fields([mp.Ex,mp.Ey,mp.Ez],
                                fcen,0,1,
                                center=mp.Vector3(),
                                size=mp.Vector3(0,Sy,Sz-2 * pml_size),
                                yee_grid=False
                                )

dft_fields_xyz12_emp = opt.sim.add_dft_fields([mp.Ex,mp.Ey,mp.Ez],
                                fcen,0,1,
                                center=mp.Vector3(0,0,0.01),
                                size=mp.Vector3(Sx,Sy,0),
                                yee_grid=False
                                )

dft_fields_xyz00_emp = opt.sim.add_dft_fields([mp.Ex,mp.Ey,mp.Ez],
                                fcen,0,1,
                                center=mp.Vector3(0,0,0),
                                size=mp.Vector3(Sx,Sy,0),
                                yee_grid=False 
                                )



dft_fields_mode_volume_emp = opt.sim.add_dft_fields([mp.Ex,mp.Ey,mp.Ez],
                                fcen,0,1,
                                center=mp.Vector3(0,0,0),
                                size=mp.Vector3(Sx,Sy,design_region_z_height + Sz_size),
                                yee_grid=False 
                                )


#opt.sim.run(until_after_sources = 0)
opt.sim.run(until_after_sources=mp.stop_when_dft_decayed(
            tol=1e-11,
            minimum_run_time=0,
            maximum_run_time=50))


# In[8]:


Ex_xz_emp = opt.sim.get_dft_array(dft_fields_xz_emp,mp.Ex,0)
Ey_xz_emp = opt.sim.get_dft_array(dft_fields_xz_emp,mp.Ey,0)
Ez_xz_emp = opt.sim.get_dft_array(dft_fields_xz_emp,mp.Ez,0)
np.save('DFT_empty_GAS/Dft_empty_GAS_Ex_xz.npy',Ex_xz_emp)
np.save('DFT_empty_GAS/Dft_empty_GAS_Ey_xz.npy',Ey_xz_emp)
np.save('DFT_empty_GAS/Dft_empty_GAS_Ez_xz.npy',Ez_xz_emp)

Ex_yz_emp = opt.sim.get_dft_array(dft_fields_yz_emp,mp.Ex,0)
Ey_yz_emp = opt.sim.get_dft_array(dft_fields_yz_emp,mp.Ey,0)
Ez_yz_emp = opt.sim.get_dft_array(dft_fields_yz_emp,mp.Ez,0)
np.save('DFT_empty_GAS/Dft_empty_GAS_Ex_yz.npy',Ex_yz_emp)
np.save('DFT_empty_GAS/Dft_empty_GAS_Ey_yz.npy',Ey_yz_emp)
np.save('DFT_empty_GAS/Dft_empty_GAS_Ez_yz.npy',Ez_yz_emp)

Ex01_emp = opt.sim.get_dft_array(dft_fields_xyz12_emp,mp.Ex,0)
Ey01_emp = opt.sim.get_dft_array(dft_fields_xyz12_emp,mp.Ey,0)
Ez01_emp = opt.sim.get_dft_array(dft_fields_xyz12_emp,mp.Ez,0)
np.save('DFT_empty_GAS/Dft_empty_GAS_Ex_xyz1.npy',Ex01_emp)
np.save('DFT_empty_GAS/Dft_empty_GAS_Ey_xyz1.npy',Ey01_emp)
np.save('DFT_empty_GAS/Dft_empty_GAS_Ez_xyz1.npy',Ez01_emp)

Ex00_emp = opt.sim.get_dft_array(dft_fields_xyz00_emp,mp.Ex,0)
Ey00_emp = opt.sim.get_dft_array(dft_fields_xyz00_emp,mp.Ey,0)
Ez00_emp = opt.sim.get_dft_array(dft_fields_xyz00_emp,mp.Ez,0)
np.save('DFT_empty_GAS/Dft_empty_GAS_Ex_xyz0.npy',Ex00_emp)
np.save('DFT_empty_GAS/Dft_empty_GAS_Ey_xyz0.npy',Ey00_emp)
np.save('DFT_empty_GAS/Dft_empty_GAS_Ez_xyz0.npy',Ez00_emp)

Ex_mode_volume_emp = opt.sim.get_dft_array(dft_fields_mode_volume_emp,mp.Ex,0)
Ey_mode_volume_emp = opt.sim.get_dft_array(dft_fields_mode_volume_emp,mp.Ey,0)
Ez_mode_volume_emp = opt.sim.get_dft_array(dft_fields_mode_volume_emp,mp.Ez,0)
np.save('DFT_mode_volume_emp/DFT_mode_volume_emp_Ex_xyz0.npy',Ex_mode_volume_emp)
np.save('DFT_mode_volume_emp/DFT_mode_volume_emp_Ey_xyz0.npy',Ey_mode_volume_emp)
np.save('DFT_mode_volume_emp/DFT_mode_volume_emp_Ez_xyz0.npy',Ez_mode_volume_emp)